### Imports

In [1]:
import sqlite3
import warnings

import pandas as pd
from pandas.core.common import SettingWithCopyWarning
import altair as alt

In [2]:
# This will remove unnecessary warnings that jupyter always rise
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

### Constants

In [12]:
DB_FILE = 'Data scientist exercise.db'
DB_SELECT = 'SELECT * from access_log'

# Column names
EVENT_TYPE = 'event_type'
REVENUE = 'revenue'
VARIANT = 'variant'
CITY = 'city'
USER_ID = 'user_id'

# Event type values
EVENT_TYPE_P_VIEW = 'property_view'
EVENT_TYPE_F_ADDED = 'property_favorite_added'
EVENT_TYPE_B_REQUEST = 'booking_request'

### Connection and data read

In [4]:
conn = sqlite3.connect(DB_FILE)
data = pd.read_sql(DB_SELECT, con=conn)

In [5]:
# Para no "ensuciar" el dataframe original y leer la bbdd todo el rato, trabajaremos con la copia por ahora
df = data.copy()

### Data Exploratory and Cleanse

Since we are interested in studying the *Booking Requests*, we will filter it now and later we will use the global dataframe to get other variables.

In [6]:
df_br = df[df[EVENT_TYPE]==EVENT_TYPE_B_REQUEST].reset_index(drop=True).copy()
df_br.head()

,datetime,user_id,variant,city,event_type,revenue
0,2021-08-01 00:50:26,556943737,A,rome,booking_request,211.234257
1,2021-08-01 05:02:36,630002484,B,madrid,booking_request,NaN
2,2021-08-01 08:39:23,741334523,B,madrid,booking_request,288.934075
3,2021-08-01 16:29:02,715418235,A,madrid,booking_request,212.030443
4,2021-08-01 16:30:38,107297881,A,rome,booking_request,175.026215


We have two groups, A/B, one is control and the other is treatment, and roll is assigned per user. Treatment group have higher fees that it is included in the shopping cart. We need to test if the **Conversion Rate** is affected by the increase in price but the extra revenue earned actually compensates it. We are going to assume that the fee charged is exactly the revenue we get.

In [7]:
print('Number of records with negative revenue for Booking Requests: {}'.format(len(df_br[df_br[REVENUE]<0])))

Number of records with negative revenue for Booking Requests: 0


In [8]:
print('Number of records for Booking Requests group A: {}'.format(len(df_br[df_br[VARIANT]=='A'])))
print('Number of records for Booking Requests group B: {}'.format(len(df_br[df_br[VARIANT]=='A'])))

Number of records for Booking Requests group A: 226
Number of records for Booking Requests group B: 226


In [9]:
df_br_na = df_br[df_br[REVENUE].isna()]
print('Number of NA records for Booking Requests: {}'.format(len(df_br_na)))
# print('Number of NA records for Booking Requests group A: {}'.format(len(df_br_na[df_br_na[VARIANT]=='A'])))
# print('Number of NA records for Booking Requests group B: {}'.format(len(df_br_na[df_br_na[VARIANT]=='B'])))
alt.Chart(df_br_na).mark_bar().encode(
    x=alt.X('count({})'.format(VARIANT)),
    y=alt.Y(VARIANT)
)

Number of NA records for Booking Requests: 16


alt.Chart(...)

In [10]:
alt.Chart(df_br_na).mark_bar().encode(
    x=alt.X('count({})'.format(CITY)),
    y=alt.Y(CITY)
)

alt.Chart(...)

After some data exploration we find out that there are 16 Booking Requests and it does not seem to be related with the city. Moreover, we find that the records are not equally distributed (as the data was), there are 7 in A group and 9 in B group. Since we prefer to mantain the equality among the number of records of each class and we do not want to *randomly* delete 2 records, we will replace all the NA values by 0.

In [11]:
df_br[REVENUE] = df_br[REVENUE].fillna(0)

We need to define now the two following metrics:
- **Conversion Rate per user (CVR)**: *Unique users that did a BR/Total Unique Users*
- **Revenue per user**: *Total Revenue Earned/Total Unique Users*
Therefore, let's get the total number of unique users and get the CVR directly calculated in the specific dataframe we have defined (*df_br*). The Revenue per user is the grouping of all the business requests by user. Since only the business requests have revenue, we can ignore the other types of revenue and continue working with the (*df_br*).

In [15]:
TOTAL_USERS = len(df[USER_ID].unique())
USERS_WITH_BR = len(df_br[USER_ID].unique())

In [16]:
USERS_WITH_BR

395

In [17]:
len(df_br)

396